# Model import using the Petab format

In this notebook, we illustrate how to use [pyPESTO](https://github.com/icb-dcm/pypesto.git) together with [PEtab](https://github.com/petab-dev/petab.git) and [AMICI](https://github.com/icb-dcm/amici.git). We employ models from the [benchmark collection](https://github.com/benchmarking-initiative/benchmark-models-petab), which we first download:

In [1]:
import pypesto
import pypesto.petab
import pypesto.optimize as optimize
import pypesto.visualize as visualize
import amici
import petab

import os
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

!git clone --depth 1 https://github.com/Benchmarking-Initiative/Benchmark-Models-PEtab.git tmp/benchmark-models || (cd tmp/benchmark-models && git pull)

folder_base = "tmp/benchmark-models/Benchmark-Models/"

fatal: Zielpfad 'tmp/benchmark-models' existiert bereits und ist kein leeres Verzeichnis.
Bereits aktuell.


## Import

### Manage PEtab model

A PEtab problem comprises all the information on the model, the data and the parameters to perform parameter estimation. We import a model as a `petab.Problem`.

In [2]:
# a collection of models that can be simulated

#model_name = "Zheng_PNAS2012"
model_name = "Boehm_JProteomeRes2014"
#model_name = "Fujita_SciSignal2010"
#model_name = "Sneyd_PNAS2002"
#model_name = "Borghans_BiophysChem1997"
#model_name = "Elowitz_Nature2000"
#model_name = "Crauste_CellSystems2017"
#model_name = "Lucarelli_CellSystems2018"
#model_name = "Schwen_PONE2014"
#model_name = "Blasi_CellSystems2016"

# the yaml configuration file links to all needed files
yaml_config = os.path.join(folder_base, model_name, model_name + '.yaml')

# create a petab problem
petab_problem = petab.Problem.from_yaml(yaml_config)

### Import model to AMICI

The model must be imported to pyPESTO and AMICI. Therefore, we create a `pypesto.PetabImporter` from the problem, and create an AMICI model.

In [3]:
importer = pypesto.petab.PetabImporter(petab_problem)

model = importer.create_model()

# some model properties
print("Model parameters:", list(model.getParameterIds()), '\n')
print("Model const parameters:", list(model.getFixedParameterIds()), '\n')
print("Model outputs:   ", list(model.getObservableIds()), '\n')
print("Model states:    ", list(model.getStateIds()), '\n')

Model parameters: ['Epo_degradation_BaF3', 'k_exp_hetero', 'k_exp_homo', 'k_imp_hetero', 'k_imp_homo', 'k_phos', 'ratio', 'specC17', 'noiseParameter1_pSTAT5A_rel', 'noiseParameter1_pSTAT5B_rel', 'noiseParameter1_rSTAT5A_rel'] 

Model const parameters: [] 

Model outputs:    ['pSTAT5A_rel', 'pSTAT5B_rel', 'rSTAT5A_rel'] 

Model states:     ['STAT5A', 'STAT5B', 'pApB', 'pApA', 'pBpB', 'nucpApA', 'nucpApB', 'nucpBpB'] 



### Create objective function

To perform parameter estimation, we need to define an objective function, which integrates the model, data, and noise model defined in the PEtab problem.

In [4]:
import libsbml
converter_config = libsbml.SBMLLocalParameterConverter()\
    .getDefaultProperties()
petab_problem.sbml_document.convert(converter_config)

obj = importer.create_objective()

# for some models, hyperparamters need to be adjusted
#obj.amici_solver.setMaxSteps(10000)
#obj.amici_solver.setRelativeTolerance(1e-7)
#obj.amici_solver.setAbsoluteTolerance(1e-7)

We can request variable derivatives via `sensi_orders`, or function values or residuals as specified via `mode`. Passing `return_dict`, we obtain the direct result of the AMICI simulation.

In [5]:
ret = obj(petab_problem.x_nominal_scaled, mode='mode_fun', sensi_orders=(0,1), return_dict=True)
print(ret)

{'fval': 138.2219980325784, 'grad': array([ 2.20274551e-02,  5.53227528e-02,  5.78848630e-03,  5.39469979e-03,
       -4.51595807e-05,  7.91352044e-03,  0.00000000e+00,  1.07805893e-02,
        2.40364901e-02,  1.91910649e-02,  0.00000000e+00]), 'rdatas': [<amici.numpy.ReturnDataView object at 0x7f60dbffd340>]}


The problem defined in PEtab also defines the fixing of parameters, and parameter bounds. This information is contained in a `pypesto.Problem`.

In [6]:
problem = importer.create_problem(obj)

In particular, the problem accounts for the fixing of parametes.

In [7]:
print(problem.x_fixed_indices, problem.x_free_indices)

[6, 10] [0, 1, 2, 3, 4, 5, 7, 8, 9]


The problem creates a copy of he objective function that takes into account the fixed parameters. The objective function is able to calculate function values and derivatives. A finite difference check whether the computed gradient is accurate:

In [8]:
objective = problem.objective
ret = objective(petab_problem.x_nominal_free_scaled, sensi_orders=(0,1))
print(ret)

(138.2219980325784, array([ 2.20274551e-02,  5.53227528e-02,  5.78848630e-03,  5.39469979e-03,
       -4.51595807e-05,  7.91352044e-03,  1.07805893e-02,  2.40364901e-02,
        1.91910649e-02]))


In [9]:
eps = 1e-4

def fd(x):
    grad = np.zeros_like(x)
    j = 0
    for i, xi in enumerate(x):
        mask = np.zeros_like(x)
        mask[i] += eps
        valinc, _ = objective(x+mask, sensi_orders=(0,1))
        valdec, _ = objective(x-mask, sensi_orders=(0,1))
        grad[j] = (valinc - valdec) / (2*eps)
        j += 1
    return grad

fdval = fd(petab_problem.x_nominal_free_scaled)
print("fd: ", fdval)
print("l2 difference: ", np.linalg.norm(ret[1] - fdval))

fd:  [0.01251672 0.05435043 0.01127779 0.00407249 0.00251655 0.00484676
 0.01077929 0.02403519 0.01918976]
l2 difference:  0.011800321295168533


### In short

All of the previous steps can be shortened by directly creating an importer object and then a problem:

In [10]:
importer = pypesto.petab.PetabImporter.from_yaml(yaml_config)
problem = importer.create_problem()

## Run optimization

Given the problem, we can perform optimization. We can specify an optimizer to use, and a parallelization engine to speed things up.

In [ ]:
optimizer = optimize.ScipyOptimizer()

# engine = pypesto.engine.SingleCoreEngine()
engine = pypesto.engine.MultiProcessEngine()

# do the optimization
result = optimize.minimize(problem=problem, optimizer=optimizer,
                           n_starts=10, engine=engine)

Engine set up to use up to 8 processes in total. The number was automatically determined and might not be appropriate on some systems.
[Warning] AMICI:CVODES:CVode:ERR_FAILURE: AMICI ERROR: in module CVODES in function CVode : At t = 94.0005 and h = 1.64835e-05, the error test failed repeatedly or with |h| = hmin. 
[Warning] AMICI:simulation: AMICI forward simulation failed at t = 94.000459:
AMICI failed to integrate the forward problem

[Warning] AMICI:CVODES:CVode:ERR_FAILURE: AMICI ERROR: in module CVODES in function CVode : At t = 94.0005 and h = 1.64835e-05, the error test failed repeatedly or with |h| = hmin. 
[Warning] AMICI:simulation: AMICI forward simulation failed at t = 94.000459:
AMICI failed to integrate the forward problem



## Predict

### Create prediction function

We can use pyPESTO for predictions: Any function of the model observables can be computed by using a post-processing function, which can be passed to the model prediction.

In [ ]:
# create a prediction function, which approximates, e.g., integral over time of an observable.

def post_process_y(ys_list: list):
    # we will need the output timepoints in this example
    timepoints = [0., 2.5, 5., 10., 15., 20., 30., 40., 50., 60., 80., 100., 120., 160., 200., 240.]

    # define outputs
    outputs = []
    # iterate over simulation conditions (in this example, there is only one)
    for ys in ys_list:
        # In this example, we're only interested in the first observable
        y = list(ys[:,0])
        integral = [0.,]
        for i, iy in enumerate(y[:-1]):
            integral.append(integral[-1] + (timepoints[i+1] - timepoints[i]) * (y[i+1] + iy) / 2)
        
        # add the integral to the outputs (one output per condition)
        outputs.append(integral)
        
    return outputs

pred = importer.create_prediction(obj, 
                                  post_processing=post_process_y,
                                  observables=['y0_integral',])

We can request variable derivatives via `sensi_orders` also for predictions.

In [ ]:
ret = pred(petab_problem.x_nominal_scaled, sensi_orders=(0,1))
print('timepoints:', ret.conditions[0].timepoints)
print('observables:', ret.conditions[0].observables)
print('output_', ret.conditions[0].output)

## Visualize

The results are contained in a `pypesto.Result` object. It contains e.g. the optimal function values.

In [ ]:
result.optimize_result.get_for_key('fval')

We can use the standard pyPESTO plotting routines to visualize and analyze the results.

In [ ]:
ref = visualize.create_references(
    x=petab_problem.x_nominal_scaled, fval=obj(petab_problem.x_nominal_scaled))

visualize.waterfall(result, reference=ref, scale_y='lin')
visualize.parameters(result, reference=ref)